In [5]:
from openai import OpenAI 
import json 

from dotenv import load_dotenv, find_dotenv

_: bool = load_dotenv(find_dotenv())

client : OpenAI = OpenAI()

In [9]:
# example dummy function hard coded to return the same weather
# in production, this could be your backend API or an external API

def get_current_weather(location:str, unit:str="fahrenheit")->str:
    '''get the cureent weather in a given location'''
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temprerature": "10", "unit": "celsius"})
    elif "san franciso" in location.lower():
        return json.dumps({"location": "San Franciso", "temprerature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temprerature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temprerature": "unknown"})

In [10]:
from openai.types.chat.chat_completion import ChatCompletion , ChatCompletion, ChatCompletionMessage

def run_conversation(main_request: str)->str:

    messages = [{"role": "user", "content": main_request}]
    # Step 1: send the  conversation and available functions to the model 
    tools = [
        {
            "type": "function",
            "function":{
                "name": "get_current_weather",
                "description": "get the cureent weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Franciso, CA "
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"]
                        }
                    },
                    "required": ["location"]
                },
            },
        }
    ]


# First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages= messages,
        tools= tools,
    tool_choice="auto", # auto is default , but we'll be explicit
)
    
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response:", dict(response_message))

    tool_calls = response_message.tool_calls
    display("First Response Tool Calls:", list(tool_calls))



    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may be not always be valid; be sure to handles error
        available_functions = {
            "get_current_weather": get_current_weather,
        } # only one function in the example, but you can have multiple

        messages.append(response_message) # extend conversation with assistant's reply

    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )
    display("* Second Request Message:", list(messages))
    second_response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages= messages,        
    )

    print("* Second Response:", dict(second_response)) 
    return second_response.choices[0].message.content



In [11]:
run_conversation("what is the weather like in San Francisco, Tokyo, and Paris?")

'* First Response:'

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_tN8OybHrukJ7VjHe89klYdyy', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_nrhqPrFhH4b4FkGtONvFVu9D', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_XCvcE8b4K5bh5JT8Hxgp96Sp', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'First Response Tool Calls:'

[ChatCompletionMessageToolCall(id='call_tN8OybHrukJ7VjHe89klYdyy', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_nrhqPrFhH4b4FkGtONvFVu9D', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_XCvcE8b4K5bh5JT8Hxgp96Sp', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Second Request Message:'

[{'role': 'user',
  'content': 'what is the weather like in San Francisco, Tokyo, and Paris?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_tN8OybHrukJ7VjHe89klYdyy', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nrhqPrFhH4b4FkGtONvFVu9D', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_XCvcE8b4K5bh5JT8Hxgp96Sp', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_tN8OybHrukJ7VjHe89klYdyy',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temprerature": "unknown"}'},
 {'tool_call_id': 'call_nrhqPrFhH4b4FkGtONvFVu9D',
  'role': 'tool',
  'name

* Second Response: {'id': 'chatcmpl-8dIdi0l5pSRAcGe6Y8d6VH93L7Nfj', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Currently, it is unknown what the temperature is in San Francisco. In Tokyo, the temperature is 10°C, and in Paris, the temperature is 22°C.', role='assistant', function_call=None, tool_calls=None))], 'created': 1704377134, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_99cc374e39', 'usage': CompletionUsage(completion_tokens=34, prompt_tokens=171, total_tokens=205)}


'Currently, it is unknown what the temperature is in San Francisco. In Tokyo, the temperature is 10°C, and in Paris, the temperature is 22°C.'

# panaverse github Code

In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()


In [2]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location:str, unit:str="fahrenheit")->str:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [3]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content


In [4]:
run_conversation("What's the weather like in San Francisco, Tokyo, and Paris?")


'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_icKgSzL0FxwhllsQKDb3nNHV', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_xiAViE7eY1kyYR7qyfNEjuQY', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_vtI4caosS0GNtrmag6oJfO9d', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_icKgSzL0FxwhllsQKDb3nNHV', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_xiAViE7eY1kyYR7qyfNEjuQY', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_vtI4caosS0GNtrmag6oJfO9d', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Second Request Messages: '

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_icKgSzL0FxwhllsQKDb3nNHV', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_xiAViE7eY1kyYR7qyfNEjuQY', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_vtI4caosS0GNtrmag6oJfO9d', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_icKgSzL0FxwhllsQKDb3nNHV',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_xiAViE7eY1kyYR7qyfNEjuQY',
  'role': 

* Second Response:  {'id': 'chatcmpl-8dHMQylGL2JbeKS2Ryf65yJmitxzw', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather is as follows:\n- San Francisco: 72°F\n- Tokyo: 10°C\n- Paris: 22°C', role='assistant', function_call=None, tool_calls=None))], 'created': 1704372218, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_99cc374e39', 'usage': CompletionUsage(completion_tokens=28, prompt_tokens=169, total_tokens=197)}


'The current weather is as follows:\n- San Francisco: 72°F\n- Tokyo: 10°C\n- Paris: 22°C'